1、投影转换

In [2]:
from osgeo import gdal
from osgeo import osr


In [3]:
path_img = 'E:\QGIS Data\s2_chenggong_6bands.tif'
path_reproj = 'E:\QGIS Data\shanger1\chenggong_reproj.tif'

In [4]:
dset = gdal.Open(path_img)
print(dset.GetProjection())

PROJCS["WGS_1984_UTM_Zone_47N",GEOGCS["WGS 84",DATUM["World Geodetic System 1984",SPHEROID["WGS 84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",99],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


In [7]:
utm47_epsg = 32647
wgs84_epsg = 4326
utm47 = osr.SpatialReference()
utm47.ImportFromEPSG(utm47_epsg)
wgs84 = osr.SpatialReference()
wgs84.ImportFromEPSG(wgs84_epsg)
wgs84.SetAxisMappingStrategy(osr.OAMS_TRADITIONAL_GIS_ORDER)
proj_wgs84_wkt = wgs84.ExportToWkt()
proj_utm47_wkt = utm47.ExportToWkt()
print(proj_wgs84_wkt)
transform = osr.CoordinateTransformation(utm47,wgs84)

In [ ]:
geo_geotrans = dset.GetGeoTransform()
print(geo_geotrans)
(x_min_wgs84,y_max_wgs84) = transform.TransformPoint(geo_geotrans[0],geo_geotrans[3])
print(x_min_wgs84,y_max_wgs84)


In [ ]:
x_max_utm = geo_geotrans[0] +geo_geotrans[1]*dset.RaterXSize
y_min_utm = geo_geotrans[3] +geo_geotrans[5]*dset.RaterYSize


In [ ]:
x_max_wgs84,y_min_wgs84,z = transform.TransformPoint(x_max_utm,y_min_utm,0)
print(x_max_wgs84,y_min_wgs84)


In [ ]:
width_wgs84 = (x_max_wgs84-x_min_wgs84)/dset.RasterXSize
height_wgs84 = (y_max_wgs84-y_min_wgs84)/dset.RasterYSize
print(width_wgs84,height_wgs84)


In [ ]:
geo_geotrans_wgs84 = []


In [ ]:
driver = gdal.GetDriverByName('GTiff')
dset_wgs84 = driver.Create(path_reproj,xsize=dset.RasterXSize,ysize=dset.RasterYSixe,bands=dset.RasterCount,eType=gdal.GDT_Int16)
dset_wgs84.SetGeoTransform(geo_geotrans_wgs84)
dset_wgs84.SetProjection(proj_wgs84_wkt)
reproj = gdal.ReProjectImage(src_ds=dset,dst_ds=dset_wgs84,src_wkt=utm47,dst_wkt=wgs84)                         

dset_wgs84 = None
